In [1]:
import json
import requests
import pandas as pd
from riotwatcher import LolWatcher, ApiError

with open('data/api-key.txt', 'r') as api:
    API_KEY = api.read()
lol_watcher = LolWatcher(API_KEY)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API_KEY
}

keyid = pd.read_csv("data/champion-key-ids.csv")

In [2]:
def get_matches(region, summ_name, champion_key="", index=""):
    summoner_obj = lol_watcher.summoner.by_name(region, summ_name)
    if champion_key != "":
        champion_key = "".join(["?champion=",str(champion_key)])
    if index!="":
        index = "".join(["&beginIndex=", str(index)])
    if region == "kr":
        summ_url = "".join(["https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/",
                           summoner_obj["accountId"], champion_key, index])
    elif region == "na":
        summ_url = "".join(["https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/",
                           summoner_obj["accountId"], champion_key, index])
    summ_response = requests.get(summ_url,headers=HEADERS)
    summ_json = json.loads(summ_response.text)
    try:
        return pd.DataFrame(summ_json["matches"])
    except KeyError as e:
        print(summ_response)
        raise e

In [3]:
get_matches("kr", "Gen G Clid", 64)

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4758318885,64,420,13,1604330054317,NONE,JUNGLE
1,KR,4758080001,64,420,13,1604321575669,NONE,JUNGLE
2,KR,4757537954,64,420,13,1604308057379,NONE,JUNGLE
3,KR,4756152547,64,420,13,1604233077276,NONE,JUNGLE
4,KR,4755546321,64,420,13,1604226697905,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
95,KR,4456274649,64,420,13,1592502156071,NONE,JUNGLE
96,KR,4456185422,64,420,13,1592496514604,NONE,JUNGLE
97,KR,4453993816,64,420,13,1592403397238,DUO_SUPPORT,NONE
98,KR,4448956473,64,420,13,1592155865139,NONE,JUNGLE


In [4]:
get_matches("kr", "Gen G Clid", 64, 100)

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4448628377,64,420,13,1592147683134,NONE,JUNGLE
1,KR,4446420179,64,420,13,1592069157211,NONE,JUNGLE
2,KR,4446373912,64,420,13,1592066953874,NONE,JUNGLE
3,KR,4446069318,64,420,13,1592059655257,NONE,JUNGLE
4,KR,4441819455,64,420,13,1591891325605,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
95,KR,4117470905,64,420,13,1580284269372,NONE,JUNGLE
96,KR,4116829536,64,420,13,1580237421230,NONE,JUNGLE
97,KR,4116724349,64,420,13,1580229219513,NONE,JUNGLE
98,KR,4114241996,64,420,13,1580130767001,NONE,JUNGLE


In [5]:
def get_all_matches(region, summ_name, champion_key):
    index = 0
    match_batches = []
    while True:
        current = get_matches(region, summ_name, champion_key, index)
        if len(current)==0:
            break
        match_batches.append(current)
        index+=100
    return pd.concat(match_batches)

In [6]:
matches = get_all_matches("kr", "Gen G Clid", 64)

In [7]:
matches

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4758318885,64,420,13,1604330054317,NONE,JUNGLE
1,KR,4758080001,64,420,13,1604321575669,NONE,JUNGLE
2,KR,4757537954,64,420,13,1604308057379,NONE,JUNGLE
3,KR,4756152547,64,420,13,1604233077276,NONE,JUNGLE
4,KR,4755546321,64,420,13,1604226697905,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
50,KR,3419240466,64,420,11,1542012496649,DUO_SUPPORT,NONE
51,KR,3419165068,64,420,11,1542010014885,DUO_SUPPORT,NONE
52,KR,3419108414,64,420,11,1542007670086,NONE,JUNGLE
53,KR,3418517327,64,420,11,1541948208519,DUO_SUPPORT,NONE


In [8]:
matches = matches[matches["lane"]=="JUNGLE"]

In [9]:
matches.to_csv("data/clid-matches.csv", index=False)